Implementation of Byte pair encoding from scratch [Andrej Karpathy video]

A Good tokenization webapp : https://tiktokenizer.vercel.app

Q: -**What is string in python?**

A:- A string are immutable sequences of Unicode code points.  


So what are unicode code points?

It is text encoding standard maintained by Unicode Consortium designed to support the use of text written in all world's major writing systems.  
 It supports 148813 characters and 161 scripts
 

In [8]:
#unicode code point access by ord function in python

ord("h")

104

In [9]:
ord("👋")

128075

In [12]:
ord("€")

8364

In [13]:
[ord(x) for x in "hello 👋, how are you doing?"]

[104,
 101,
 108,
 108,
 111,
 32,
 128075,
 44,
 32,
 104,
 111,
 119,
 32,
 97,
 114,
 101,
 32,
 121,
 111,
 117,
 32,
 100,
 111,
 105,
 110,
 103,
 63]

In [14]:
list("hello 👋, how are you doing?".encode("utf-8"))

[104,
 101,
 108,
 108,
 111,
 32,
 240,
 159,
 145,
 139,
 44,
 32,
 104,
 111,
 119,
 32,
 97,
 114,
 101,
 32,
 121,
 111,
 117,
 32,
 100,
 111,
 105,
 110,
 103,
 63]

Q:- **why can't we simply use unicode code points and not tokenization ?**

1. It can make vocabulary quite long of 148813 code points.
2. It is very much alive and not stable 

We need something better and here comes encodings and there are 3 types of encodings

1. UTF-8
2. UTF-16
3. UTF-32


These encodings takes unicode text and translate into binary data or byte streams.

> UTF-8 is the most common


Disadvantage of using UTF-8 is it imply vocab length of only 256 tokens.Also it makes all our text stretch out very long sequence of bytes. So it is very inefficient for next token prediction task.


So we don't want to use raw byte of UTF-8, we want to support larger vocab size but stick to UTF-8 encodings of these strings. So answer is Byte Pair encodings which allows us to compress these byte sequences.

## Byte Pair Encoding 

Lets say we have vocab size of 4 and has following sequnence of length 11.

`aaabdaaabac`  

Now what we do is to iteratively find pair of tokens which occurs most frequently, and once identified that pair we replace with a single new token

`ZabdZabac`  
Z=aa

then repeat the process with `ab`  
`ZYdZYac`  
Y=ab  
Z=aa  

again look at sequence and replace ZY with X   
`XdXac`  
X=ZY  
Y=ab  
Z=aa  



Now we are left with sequence of length 5  and vocab of size 7   

In similar manner we r going to mint new tokens from 256 byte sequences  and adding upto our vocabulary.

Let's implement this !!!

In [30]:
text="""
Apollo 11 was a spaceflight conducted by the United States from July 16 to July 24, 1969. It marked the first time in history that humans landed on the Moon. Commander Neil Armstrong 
and Lunar Module Pilot Buzz Aldrin landed the Apollo Lunar Module Eagle on July 20, 1969, at 20:17 UTC, and Armstrong became the first person to step onto the Moon's surface six hours 
and 39 minutes later, on July 21 at 02:56 UTC. Aldrin joined him 19 minutes later, and they spent about two and a quarter hours together exploring the site they had named Tranquility Base 
upon landing. Armstrong and Aldrin collected 47.5 pounds (21.5 kg) of lunar material to bring back to Earth as pilot Michael Collins flew the Command Module Columbia in lunar orbit, and were on 
the Moon's surface for 21 hours, 36 minutes, before lifting off to rejoin Columbia.
"""

tokens = text.encode("utf-8")
tokens = list(map(int, tokens))

print("---")
print(text)
print("length:", len(text))
print("---")
print(tokens)

print("length:",len(tokens))

---

Apollo 11 was a spaceflight conducted by the United States from July 16 to July 24, 1969. It marked the first time in history that humans landed on the Moon. Commander Neil Armstrong 
and Lunar Module Pilot Buzz Aldrin landed the Apollo Lunar Module Eagle on July 20, 1969, at 20:17 UTC, and Armstrong became the first person to step onto the Moon's surface six hours 
and 39 minutes later, on July 21 at 02:56 UTC. Aldrin joined him 19 minutes later, and they spent about two and a quarter hours together exploring the site they had named Tranquility Base 
upon landing. Armstrong and Aldrin collected 47.5 pounds (21.5 kg) of lunar material to bring back to Earth as pilot Michael Collins flew the Command Module Columbia in lunar orbit, and were on 
the Moon's surface for 21 hours, 36 minutes, before lifting off to rejoin Columbia.

length: 838
---
[10, 65, 112, 111, 108, 108, 111, 32, 49, 49, 32, 119, 97, 115, 32, 97, 32, 115, 112, 97, 99, 101, 102, 108, 105, 103, 104, 116, 32, 99, 111,

In [36]:
# find pair of bytes that occurred most frequently !

def get_stats(ids):
    counts = {}

    for pair in zip(ids, ids[1:]):
        counts[pair] = counts.get(pair,0)+1
    return counts

stats = get_stats(tokens)

# print(stats)
# print(sorted(((v,k) for k,v in stats.items()), reverse=True))

In [37]:
top_pair = max(stats, key=stats.get)
top_pair

(101, 32)

In [39]:
def merge(ids, pair, idx):
    newids=[]
    i = 0
    while i < len(ids):
        if i < len(ids) - 1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
            newids.append(idx)
            i+=2
        else:
            newids.append(ids[i])
            i += 1
    return newids
    
tokens2 = merge(tokens, top_pair, 256)
print(tokens2)
print("length:", len(tokens2))

[10, 65, 112, 111, 108, 108, 111, 32, 49, 49, 32, 119, 97, 115, 32, 97, 32, 115, 112, 97, 99, 101, 102, 108, 105, 103, 104, 116, 32, 99, 111, 110, 100, 117, 99, 116, 101, 100, 32, 98, 121, 32, 116, 104, 256, 85, 110, 105, 116, 101, 100, 32, 83, 116, 97, 116, 101, 115, 32, 102, 114, 111, 109, 32, 74, 117, 108, 121, 32, 49, 54, 32, 116, 111, 32, 74, 117, 108, 121, 32, 50, 52, 44, 32, 49, 57, 54, 57, 46, 32, 73, 116, 32, 109, 97, 114, 107, 101, 100, 32, 116, 104, 256, 102, 105, 114, 115, 116, 32, 116, 105, 109, 256, 105, 110, 32, 104, 105, 115, 116, 111, 114, 121, 32, 116, 104, 97, 116, 32, 104, 117, 109, 97, 110, 115, 32, 108, 97, 110, 100, 101, 100, 32, 111, 110, 32, 116, 104, 256, 77, 111, 111, 110, 46, 32, 67, 111, 109, 109, 97, 110, 100, 101, 114, 32, 78, 101, 105, 108, 32, 65, 114, 109, 115, 116, 114, 111, 110, 103, 32, 10, 97, 110, 100, 32, 76, 117, 110, 97, 114, 32, 77, 111, 100, 117, 108, 256, 80, 105, 108, 111, 116, 32, 66, 117, 122, 122, 32, 65, 108, 100, 114, 105, 110, 32, 108

In [40]:
text = """Apollo 11 was a spaceflight conducted by the United States from July 16 to July 24, 1969. It marked the first time in history that humans landed on the Moon. Commander Neil Armstrong and Lunar Module Pilot Buzz Aldrin landed the Apollo Lunar Module Eagle on July 20, 1969, at 20:17 UTC, and Armstrong became the first person to step onto the Moon's surface six hours and 39 minutes later, on July 21 at 02:56 UTC. Aldrin joined him 19 minutes later, and they spent about two and a quarter hours together exploring the site they had named Tranquility Base upon landing. Armstrong and Aldrin collected 47.5 pounds (21.5 kg) of lunar material to bring back to Earth as pilot Michael Collins flew the Command Module Columbia in lunar orbit, and were on the Moon's surface for 21 hours, 36 minutes, before lifting off to rejoin Columbia.

Apollo 11 was launched by a Saturn V rocket from Kennedy Space Center on Merritt Island, Florida, on July 16 at 13:32 UTC, and it was the fifth crewed mission of NASA's Apollo program. The Apollo spacecraft had three parts: a command module (CM) with a cabin for the three astronauts, the only part that returned to Earth; a service module (SM), which supported the command module with propulsion, electrical power, oxygen, and water; and a lunar module (LM) that had two stages—a descent stage for landing on the Moon and an ascent stage to place the astronauts back into lunar orbit.

After being sent to the Moon by the Saturn V's third stage, the astronauts separated the spacecraft from it and traveled for three days until they entered lunar orbit. Armstrong and Aldrin then moved into Eagle and landed in the Sea of Tranquility on July 20. The astronauts used Eagle's ascent stage to lift off from the lunar surface and rejoin Collins in the command module. They jettisoned Eagle before they performed the maneuvers that propelled Columbia out of the last of its 30 lunar orbits onto a trajectory back to Earth.[9] They returned to Earth and splashed down in the Pacific Ocean on July 24 after more than eight days in space.

Armstrong's first step onto the lunar surface was broadcast on live TV to a worldwide audience. He described the event as "one small step for [a] man, one giant leap for mankind."[a][15] Apollo 11 effectively proved U.S. victory in the Space Race to demonstrate spaceflight superiority, by fulfilling a national goal proposed in 1961 by President John F. Kennedy, "before this decade is out, of landing a man on the Moon and returning him safely to the Earth."[16]

Background
In the late 1950s and early 1960s, the United States was engaged in the Cold War, a geopolitical rivalry with the Soviet Union.[17] On October 4, 1957, the Soviet Union launched Sputnik 1, the first artificial satellite. This surprise success fired fears and imaginations around the world. It demonstrated that the Soviet Union had the capability to deliver nuclear weapons over intercontinental distances, and challenged American claims of military, economic, and technological superiority.[18] This precipitated the Sputnik crisis, and triggered the Space Race to prove which superpower would achieve superior spaceflight capability.[19] President Dwight D. Eisenhower responded to the Sputnik challenge by creating the National Aeronautics and Space Administration (NASA), and initiating Project Mercury,[20] which aimed to launch a man into Earth orbit.[21] But on April 12, 1961, Soviet cosmonaut Yuri Gagarin became the first person in space, and the first to orbit the Earth.[22] Nearly a month later, on May 5, 1961, Alan Shepard became the first American in space, completing a 15-minute suborbital journey. After being recovered from the Atlantic Ocean, he received a congratulatory telephone call from Eisenhower's successor, John F. Kennedy.[23]

Since the Soviet Union had higher lift capacity launch vehicles, Kennedy chose, from among options presented by NASA, a challenge beyond the capacity of the existing generation of rocketry, so that the US and Soviet Union would be starting from a position of equality. A crewed mission to the Moon would serve this purpose.[24]

On May 25, 1961, Kennedy addressed the United States Congress on "Urgent National Needs" and declared:

I believe that this nation should commit itself to achieving the goal, before this decade [1960s] is out, of landing a man on the Moon and returning him safely to the Earth. No single space project in this period will be more impressive to mankind, or more important for the long-range exploration of space; and none will be so difficult or expensive to accomplish. We propose to accelerate the development of the appropriate lunar space craft. We propose to develop alternate liquid and solid fuel boosters, much larger than any now being developed, until certain which is superior. We propose additional funds for other engine development and for unmanned explorations—explorations which are particularly important for one purpose which this nation will never overlook: the survival of the man who first makes this daring flight. But in a very real sense, it will not be one man going to the Moon—if we make this judgment affirmatively, it will be an entire nation. For all of us must work to put him there.

— Kennedy's speech to Congress[25]
On September 12, 1962, Kennedy delivered another speech before a crowd of about 40,000 people in the Rice University football stadium in Houston, Texas.[26][27] A widely quoted refrain from the middle portion of the speech reads as follows:

There is no strife, no prejudice, no national conflict in outer space as yet. Its hazards are hostile to us all. Its conquest deserves the best of all mankind, and its opportunity for peaceful cooperation may never come again. But why, some say, the Moon? Why choose this as our goal? And they may well ask, why climb the highest mountain? Why, 35 years ago, fly the Atlantic? Why does Rice play Texas? We choose to go to the Moon! We choose to go to the Moon ... We choose to go to the Moon in this decade and do the other things, not because they are easy, but because they are hard; because that goal will serve to organize and measure the best of our energies and skills, because that challenge is one that we are willing to accept, one we are unwilling to postpone, and one we intend to win, and the others, too.[28]""" 


tokens = text.encode("utf-8")
tokens = list(map(int, tokens))

In [41]:
def get_stats(ids):
    counts = {}

    for pair in zip(ids, ids[1:]):
        counts[pair] = counts.get(pair,0)+1
    return counts

def merge(ids, pair, idx):
    newids=[]
    i = 0
    while i < len(ids):
        if i < len(ids) - 1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
            newids.append(idx)
            i+=2
        else:
            newids.append(ids[i])
            i += 1
    return newids


vocab_size = 276
num_merges = vocab_size - 256
ids = list(tokens)

merges ={}

for i in range(num_merges):
    stats = get_stats(ids)
    pair = max(stats, key=stats.get)

    idx = 256 +i
    print(f"merging {pair} into a new token {idx}")
    ids  = merge(ids, pair,idx)
    merges[pair] = idx
    
# tokens2 = merge(tokens, top_pair, 256)
# print(tokens2)
# print("length:", len(tokens2))



merging (101, 32) into a new token 256
merging (116, 104) into a new token 257
merging (100, 32) into a new token 258
merging (111, 110) into a new token 259
merging (116, 32) into a new token 260
merging (97, 110) into a new token 261
merging (115, 32) into a new token 262
merging (257, 256) into a new token 263
merging (105, 110) into a new token 264
merging (44, 32) into a new token 265
merging (101, 114) into a new token 266
merging (111, 32) into a new token 267
merging (121, 32) into a new token 268
merging (111, 114) into a new token 269
merging (97, 114) into a new token 270
merging (116, 267) into a new token 271
merging (101, 258) into a new token 272
merging (261, 258) into a new token 273
merging (101, 110) into a new token 274
merging (259, 32) into a new token 275


In [43]:
# Lets take a look into compression ratio

print("tokens length:", len(tokens))
print("ids length:", len(ids))
print(f"compression ratio: {len(tokens)/ len(ids):.2f}X")


tokens length: 6355
ids length: 4841
compression ratio: 1.31X


Tokenizer is a completely separate, independent module from the LLM. It has its own training set, different from LLM on which you train vocab using byte pair encoding algorithm. 


> LLM only deals with tokens and never sees any text

## decoding
Given a sequence of integers in the range [0, vocab_size] wht is the text?